In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense
import tensorflow_addons as tfa

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [2]:
np.random.seed(666)

## Current best version - 29.

In [3]:
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [4]:
def preprocess(df):
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 1, 72:2})
    del df['sig_id']
    return df

In [5]:
train = preprocess(train_features)
test = preprocess(test_features)

In [6]:
del train_targets['sig_id']

In [7]:
train_targets['cp_type'] = train['cp_type']

In [8]:
train = train[train['cp_type'] != 'ctl_vehicle']
train_targets = train_targets[train_targets['cp_type'] != 'ctl_vehicle']

train = train.drop(['cp_type'], axis=1)
train_targets = train_targets.drop(['cp_type'], axis=1)

In [9]:
train = train.reset_index().drop(['index'], axis=1)
train_targets = train_targets.reset_index().drop(['index'], axis=1)

In [10]:
train_categories = train[['cp_dose', 'cp_time']]
test_categories = test[['cp_dose', 'cp_time']]

In [11]:
test_cp_type = test['cp_type']
test = test.drop(['cp_type'], axis=1)

In [12]:
train

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,0,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,2,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,1,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,1,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,2,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,2,0,0.1608,-1.0500,0.2551,-0.2239,-0.2431,0.4256,-0.1166,-0.1777,...,0.0789,0.3538,0.0558,0.3377,-0.4753,-0.2504,-0.7415,0.8413,-0.4259,0.2434
21944,0,1,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
21945,0,1,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,-0.5565,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
21946,0,0,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,0.0463,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [13]:
def create_autoencoder():
    input_vector = Input(shape=(874,))
    encoded = Dense(3000, activation='elu')(input_vector)
    encoded = Dense(2000, activation='elu')(encoded)
    decoded = Dense(3000, activation='elu')(encoded)
    decoded = Dense(874, activation='elu')(decoded)
    
    autoencoder = tf.keras.Model(
        input_vector, 
        decoded
    )
    
    autoencoder.compile(
        optimizer='adadelta', 
        loss='mse'
    )
    
    return autoencoder

In [14]:
autoencoder = create_autoencoder()

In [15]:
autoencoder.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 874)]             0         
_________________________________________________________________
dense (Dense)                (None, 3000)              2625000   
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              6002000   
_________________________________________________________________
dense_2 (Dense)              (None, 3000)              6003000   
_________________________________________________________________
dense_3 (Dense)              (None, 874)               2622874   
Total params: 17,252,874
Trainable params: 17,252,874
Non-trainable params: 0
_________________________________________________________________


In [16]:
mu, sigma = 0, 0.05

noise = np.random.normal(
    mu, 
    sigma, 
    [21948, 874]
) 
noised_train = train + noise

In [17]:
autoencoder.fit(
    noised_train, 
    train, 
    epochs=1000,
    batch_size=128,
    shuffle=True,
    validation_split=0.2
)

Epoch 1/1000
138/138 [==============================] - 1s 10ms/step - loss: 2.0549 - val_loss: 1.9613
Epoch 2/1000
138/138 [==============================] - 1s 7ms/step - loss: 1.8969 - val_loss: 1.8440
Epoch 3/1000
138/138 [==============================] - 1s 7ms/step - loss: 1.8033 - val_loss: 1.7685
Epoch 4/1000
138/138 [==============================] - 1s 7ms/step - loss: 1.7388 - val_loss: 1.7133
Epoch 5/1000
138/138 [==============================] - 1s 7ms/step - loss: 1.6904 - val_loss: 1.6711
Epoch 6/1000
138/138 [==============================] - 1s 7ms/step - loss: 1.6532 - val_loss: 1.6385
Epoch 7/1000
138/138 [==============================] - 1s 7ms/step - loss: 1.6238 - val_loss: 1.6122
Epoch 8/1000
138/138 [==============================] - 1s 7ms/step - loss: 1.6003 - val_loss: 1.5915
Epoch 9/1000
138/138 [==============================] - 1s 8ms/step - loss: 1.5816 - val_loss: 1.5749
Epoch 10/1000
138/138 [==============================] - 1s 7ms/step - loss: 1.56

In [18]:
encoder = tf.keras.Model(
    autoencoder.input, 
    autoencoder.layers[2].output
)

In [19]:
train_features = pd.DataFrame(encoder.predict(train))
test_features = pd.DataFrame(encoder.predict(test))

In [20]:
train_features

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,-0.288209,-0.084613,0.306833,0.027031,0.794920,0.159032,-0.474173,-0.164630,0.191491,-0.663168,...,-0.401211,0.299418,-0.399510,0.168505,-0.063990,0.122390,0.361761,-0.481489,-0.295739,-0.023230
1,-0.489053,0.533348,0.561642,-0.328170,0.148360,1.066271,0.277325,0.488550,0.450038,0.572256,...,0.176148,-0.763831,0.579242,0.595223,-0.309224,0.806332,-0.011253,-0.202415,-0.051660,-0.562452
2,0.616431,0.611748,-0.199963,-0.105145,-0.427384,0.296097,1.056623,0.724954,-0.173970,0.388408,...,1.033995,-0.094299,-0.475885,-0.301483,0.979998,-0.032327,-0.224879,0.023184,-0.093343,0.650458
3,0.563504,1.198989,0.074572,0.925521,0.249227,0.461288,0.430254,0.438995,-0.571525,-0.482105,...,3.426610,0.778354,0.360439,-0.233763,1.071806,-0.631382,-0.381244,-0.551669,-0.739635,2.615823
4,-0.573161,1.223860,0.023006,-0.724670,0.164118,-0.475520,0.739778,0.254715,-0.423146,-0.297135,...,0.204772,0.329034,-0.044127,-0.113156,0.172641,-0.161572,0.636598,0.364736,0.018596,-0.620392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,-0.436869,-0.417194,-0.148951,0.316280,0.363389,-0.181390,0.008597,0.122831,0.084432,0.112713,...,0.335282,-0.426890,-0.511917,0.153921,0.150745,-0.173826,0.668973,-0.230216,-0.215054,-0.192300
21944,0.228676,0.433592,-0.168460,0.463091,0.029069,-0.411658,-0.295749,-0.394900,-0.072644,-0.025859,...,1.051152,-0.443231,-0.580335,-0.628001,-0.445149,-0.235667,-0.006335,-0.475413,-0.415014,-0.550393
21945,-0.362264,0.181122,0.032754,0.085950,0.204058,0.975244,-0.550869,0.624273,-0.059950,0.232745,...,0.205891,-0.074664,-0.021588,0.090602,0.328738,-0.402909,-0.625120,-0.741114,0.003200,0.360614
21946,1.643758,0.547342,-0.716092,-0.576418,1.537799,1.580155,0.747167,0.636528,-0.079784,0.195283,...,1.102505,1.732197,0.926786,-0.814156,-0.557487,1.610401,1.181520,-0.713562,-0.595191,-0.218694


In [21]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(2000),
        tf.keras.layers.BatchNormalization(),

        tfa.layers.WeightNormalization(tf.keras.layers.Dense(1000)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tfa.layers.WeightNormalization(tf.keras.layers.Dense(1000)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tfa.layers.WeightNormalization(tf.keras.layers.Dense(500)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tfa.layers.WeightNormalization(
            tf.keras.layers.Dense(
                206, 
                activation="sigmoid"
            )
        )
    ])
    model.compile(
        optimizer=tfa.optimizers.AdamW(
            lr=1e-3, 
            weight_decay=1e-5, 
            clipvalue=700
        ), 
        loss='binary_crossentropy'
    )
    return model

In [22]:
submission.loc[:, train_targets.columns] = 0
res = train_targets.copy()
for n, (tr, te) in enumerate(KFold(n_splits=7, random_state=666, shuffle=True).split(train_targets)):
    print(f'Fold {n}')
    
    model = create_model()
    
    model.fit(
        train_features.values[tr],
        train_targets.values[tr],
        epochs=50, 
        batch_size=128
    )
    
    submission.loc[:, train_targets.columns] += model.predict(test_features)
    res.loc[te, train_targets.columns] = model.predict(train_features.values[te])
    
submission.loc[:, train_targets.columns] /= (n+1)

metrics = []
for _target in train_targets.columns:
    metrics.append(log_loss(train_targets.loc[:, _target], res.loc[:, _target]))

Fold 0
Epoch 1/50
147/147 [==============================] - 1s 9ms/step - loss: 0.4278
Epoch 2/50
147/147 [==============================] - 1s 8ms/step - loss: 0.0458
Epoch 3/50
147/147 [==============================] - 1s 8ms/step - loss: 0.0258
Epoch 4/50
147/147 [==============================] - 2s 15ms/step - loss: 0.0221
Epoch 5/50
147/147 [==============================] - 1s 8ms/step - loss: 0.0206
Epoch 6/50
147/147 [==============================] - 1s 8ms/step - loss: 0.0196
Epoch 7/50
147/147 [==============================] - 1s 8ms/step - loss: 0.0189
Epoch 8/50
147/147 [==============================] - 1s 8ms/step - loss: 0.0183
Epoch 9/50
147/147 [==============================] - 1s 7ms/step - loss: 0.0180
Epoch 10/50
147/147 [==============================] - 1s 8ms/step - loss: 0.0176
Epoch 11/50
147/147 [==============================] - 1s 8ms/step - loss: 0.0172
Epoch 12/50
147/147 [==============================] - 1s 10ms/step - loss: 0.0169
Epoch 13/50
147/

In [23]:
print(f'OOF Metric: {np.mean(metrics)}')

OOF Metric: 0.021527009717644912


In [24]:
submission['cp_type'] = test_cp_type
for col in submission.columns:
    if col in ['sig_id', 'cp_type', 'cp_dose', 'cp_time']:
        continue
    submission.loc[submission['cp_type'] == 'ctl_vehicle', col] = 0

submission = submission.drop(['cp_type'], axis=1)

In [25]:
submission.to_csv('submission.csv', index=False)